In [15]:
# Import necessary torch and torchvision libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR10

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import optuna

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [16]:
LR = 0.003494286840772853
MOMENTUM = 0.9421216112061177
SCHEDULAR_NAME = "StepLR"
INIT_METHOD = "kaiming_normal"
REG_METHOD = "Data Augmentation + Weight Decay: 0.0005" 

In [17]:
class MyCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(MyCNN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

            nn.Flatten(), 
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes))
        self.init_weights()

    def forward(self, x):
        x = self.network(x)
        return x
    
    def init_weights(self):
            for m in self.modules(): # self.modules() iterates through all modules (layers) in the model, including nested ones, allowing for operations like weight initialization to be applied universally.
                if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                    nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
FinalModel = MyCNN().to(device)

In [18]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy,
               device: torch.device = device):
    
    accuracy.reset()
    train_loss, train_acc = 0, 0
    model.to(device)
    
    for batch, (X, y) in enumerate(data_loader):
        
        X = X.to(device)
        y = y.to(device)
        
        # Training
        model.train()
        # Forward pass
        y_pred = model(X)
        # Calculate loss per batch
        loss = loss_fn(y_pred, y)
        train_loss += loss # accumulate loss per batch
        # Update accuracy
        accuracy.update(y_pred, y)
        # Zero the gradients
        optimizer.zero_grad()
        # Backward pass
        loss.backward()
        # Update weights
        optimizer.step()
    # Loss per epoch    
    train_loss = train_loss / len(data_loader)
    train_acc = accuracy.compute()
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc*100:.2f}%")
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy,
              device: torch.device = device):
    
    
    accuracy.reset()
    ## Testing
    test_loss, test_acc = 0, 0
    # Set model to evaluation mode
    model.eval()
    # Turn off gradients
    with torch.inference_mode():
        for X, y in data_loader:
            # Move data to device
            X = X.to(device)
            y = y.to(device)
            # Forward pass
            test_pred = model(X)
            # Calculate loss per batch
            test_loss += loss_fn(test_pred, y)
            # Update accuracy
            accuracy.update(test_pred, y)
    # Loss per epoch        
    test_loss = test_loss / len(data_loader)
    # Calculate accuracy
    test_acc = accuracy.compute()
    # Print loss and accuracy per epoch
    print(f"Test loss: {test_loss:.5f}, Test acc: {test_acc*100:.2f}%\n")
    return test_loss, test_acc

In [19]:
augmentation_transform = transforms.Compose([
    #transforms.RandomResizedCrop(224),  # Crop images to 224x224
    transforms.RandomHorizontalFlip(),  # Horizontally flip images with a 50% probability
    transforms.RandomRotation(10),      # Randomly rotate images in the range (-10 degrees, 10 degrees)
    transforms.ToTensor(),              # Convert images to PyTorch tensors
])
# Download and load the CIFAR-10 dataset
train_data = CIFAR10(root='./data', 
                     train=True, 
                     download=True, 
                     transform=augmentation_transform)

test_data = CIFAR10(root='./data',
                                 train=False,
                                 download=True,
                                 transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=32,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                            batch_size=32,
                                            shuffle=False)

# See classes
class_names = train_data.classes
# Class to index
cls_to_idx = train_data.class_to_idx


Files already downloaded and verified
Files already downloaded and verified


In [20]:
# Import accuracy metric
from torchmetrics import Accuracy
accuracy = Accuracy(task="multiclass", num_classes=10).to(device)
# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(FinalModel.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

In [21]:
# Using tqdm for progress bar
from tqdm.auto import tqdm
torch.manual_seed(42)

epochs = 15

for epoch in tqdm(range(epochs)):
    
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss, train_acc = train_step(FinalModel, train_loader, loss_fn, optimizer, accuracy)
    test_loss, test_acc = test_step(FinalModel, test_loader, loss_fn, accuracy)
    scheduler.step()

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 1
-------------------------------
Train loss: 1.49068 | Train accuracy: 45.77%


  7%|▋         | 1/15 [00:32<07:35, 32.55s/it]

Test loss: 1.15696, Test acc: 58.46%

Epoch 2
-------------------------------
Train loss: 1.03306 | Train accuracy: 63.53%


 13%|█▎        | 2/15 [01:05<07:02, 32.53s/it]

...

Test loss: 0.50513, Test acc: 83.94%

Epoch 14
-------------------------------
Train loss: 0.24812 | Train accuracy: 91.44%


 93%|█████████▎| 14/15 [07:42<00:33, 33.93s/it]

Test loss: 0.50433, Test acc: 84.02%

Epoch 15
-------------------------------
Train loss: 0.23920 | Train accuracy: 91.81%


100%|██████████| 15/15 [08:14<00:00, 32.95s/it]

Test loss: 0.50683, Test acc: 83.74%

